In [ ]:
import importer
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from load_img import read_img_file

In [ ]:
MODEL_DIR = './my_cnn_model'
TRAINING_STEPS = 10000
FILELIST = [str(x) for x in ('d','b','c','a','e','f','g','h','p','j','k','l','m','n','o','i','q','r','s','t','u','v','w','x','y','z')]
#FILELIST = ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
def cnn_model_fn(features, labels, mode):
    #tf.reset_default_graph()
    input_layer = tf.reshape(features["x"], [-1,1,28,28])
    input_layer = tf.transpose(input_layer, perm=[0,2,3,1])
    # conv1_1： 神经元图， feature_map, 输出图像
    conv1_1 = tf.layers.conv2d(input_layer, 
                             28, # output channel number
                             (3,3), # kernel size
                             padding='valid',
                             activation=tf.nn.relu,
                             name='conv1_1')
    # conv1_2： 神经元图， feature_map, 输出图像
    conv1_2 = tf.layers.conv2d(conv1_1, 
                             28, # output channel number
                             (3,3), # kernel size
                             padding='valid',
                             activation=tf.nn.relu,
                             name='conv1_2')

    # 14 * 14
    pooling1 = tf.layers.max_pooling2d(conv1_2,
                                       (2,2), # kernel size
                                       (2,2), # stride
                                    name='pool1')

    # conv2_1： 神经元图、 feature_map, 输出图像
    conv2_1 = tf.layers.conv2d(pooling1, 
                             28, # output channel number
                             (3,3), # kernel size
                             padding='valid',
                             activation=tf.nn.relu,
                             name='conv2_1')
    # conv2_2： 神经元图、 feature_map, 输出图像
    conv2_2 = tf.layers.conv2d(conv2_1, 
                             28, # output channel number
                             (3,3), # kernel size
                             padding='valid',
                             activation=tf.nn.relu,
                             name='conv2_2')

    # 7 * 7
    pooling2 = tf.layers.max_pooling2d(conv2_2,
                                       (2,2), # kernel size
                                       (2,2), # stride
                                    name='pool2')

    flatten = tf.layers.flatten(pooling2)
    dense = tf.layers.dense(inputs=flatten, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout( 
        inputs=dense, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN) 
    # 全连接层,输出
    logits = tf.layers.dense(inputs=dropout, units=26)
    
    predictions = { 
       # Generate predictions (for PREDICT and EVAL mode) 
       "classes": tf.argmax(input=logits, axis=1), 
       # Add `softmax_tensor` to the graph. It is used for PREDICT and by the 
       # `logging_hook`. 
       "probabilities": tf.nn.softmax(logits, name="softmax_tensor") 
    } 
    if mode == tf.estimator.ModeKeys.PREDICT: 
     # sess = tf.InteractiveSession() 
     # print('logits:',logits.eval(session = sess)) 
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions) 
 
    # cross entropy
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    # y_ -> softmax
    # y -> one_hot
    # loss = ylogy_
    
     # Configure the Training Op (for TRAIN mode) 
    if mode == tf.estimator.ModeKeys.TRAIN: 
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001) 
        train_op = optimizer.minimize( 
            loss=loss, 
            global_step=tf.train.get_global_step()) 
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)  
    
    eval_metric_ops = { 
        "accuracy": tf.metrics.accuracy( 
            labels=labels, predictions=predictions["classes"])} 
    return tf.estimator.EstimatorSpec( 
            mode=mode, loss=loss, eval_metric_ops=eval_metric_ops) 

   # Add evaluation metrics (for EVAL mode) 

    with tf.name_scope('train_op'):
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
        #梯度下降的变种

In [ ]:
cnn_symbol_classifier = tf.estimator.Estimator( 
    model_fn=cnn_model_fn, model_dir=MODEL_DIR) 

In [ ]:
def train_cnn_model(steps):
    train_data,train_data_labels = read_img_file('train')

    # set up logging for predictions
    # log the values in the "softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

    # train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_data_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    #print(train_input_fn)
    cnn_symbol_classifier.train(
        input_fn=train_input_fn,
        steps=steps,
        hooks=[logging_hook])

def eval_cnn_model():
    eval_data, eval_data_labels = read_img_file('eval')
    # evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_data_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = cnn_symbol_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [ ]:
train_data,train_data_labels = read_img_file('train')
eval_data,eval_data_labels = read_img_file('test')

# set up logging for predictions
# log the values in the "softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

#train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
     x={"x": train_data},
     y=train_data_labels,
     batch_size=100,
     num_epochs=None,
     shuffle=True)
# print(train_input_fn)
cnn_symbol_classifier.train(
     input_fn=train_input_fn,
     steps=TRAINING_STEPS,
     hooks=[logging_hook])

# evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_data_labels,
    num_epochs=1,
    shuffle=False)
eval_results = cnn_symbol_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

In [ ]:
import cv2
from img2bin import read_img_and_convert_to_binary
from img2bin import binary_img_segment
from img2bin import normalize_matrix_value

In [ ]:
filename = '1.jpg'
img = cv2.imread(filename)
original_img, binary_img = read_img_and_convert_to_binary(img)
symbols = binary_img_segment(binary_img, original_img)
symbols_to_be_predicted = normalize_matrix_value([x['src_img'] for x in symbols])

In [ ]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn( 
     x={"x": np.array(symbols_to_be_predicted)}, 
     shuffle=False) 
predictions = cnn_symbol_classifier.predict(input_fn=predict_input_fn)

In [ ]:
def get_candidates(estimated_probabilities): 
    #print(estimated_probabilities)
    indexes_of_top_n_largest_probability = np.argsort(-estimated_probabilities)
    #print(indexes_of_top_n_largest_probability)
    candidates = [] 
    for i in range(26):
        index = indexes_of_top_n_largest_probability[i] 
        print(index)
        candidates.append({'symbol':FILELIST[index],'probability':estimated_probabilities[index]}) 
    return candidates 

In [ ]:
characters = [] 
for i,p in enumerate(predictions): 
    candidates = get_candidates(p['probabilities'])
    characters.append({'candidates':candidates}) 
    print([x['candidates'] for x in characters])

In [ ]:
for one in characters:
    print(one['candidates'][0])